In [2]:
import sys 
sys.path.append('/Users/willknott/Desktop/DIS/coursework/pds/wdk24/src')

## Generation of data

Make `generation.py` module containing `generate_from_total_pdf(N_events, f=0.1, ...)`

We generate `n` datasets at different values of `N` (size of dataset).

For a given `N`, should I run 'pilot simulations', to inform what choice for `n` is needed to get a good estimate of the probability of success. We could decide what we want the size of $\hat{\sigma_i}$ to be (roughly), as a percentage of $\hat{p_i}$, (eg. 5%) and then solve for $n_i$ in the equation:

$$\hat{\sigma_i} = \sqrt{\frac{\hat{p_i}(1-\hat{p_i})}{n_i}}$$

$$n_i = \frac{\hat{p_i}(1-\hat{p_i})}{\hat{\sigma_i}^2}$$

Suppose we want the uncertainty to be ~5% of our estimate: $\hat{\sigma_i} = 0.05\hat{p_i}$. Then using the equation above, we get:

$$n_i = 400(\frac{1}{\hat{p_i}} - 1)$$

For $\hat{\sigma_i} = k\hat{p_i}$, we get

$$n_i = \frac{1}{k^2}(\frac{1}{\hat{p_i}} - 1)$$

We could start off with $n_i = 20$ to get our estimate $\hat{p_i}$, then use this to get a better $n_i$

## Hypothesis test on each dataset

Use Matt's example for the higgs discovery in lecture 16 as guidance.

We fit the background only model and total model to the dataset, using MLE (or another estimation method).

Calculate the neyman-pearson test statistic (I think `iminuit` can do this automatically).

Assume: this test statistic, given the null hypothesis, is chi squared distributed with k=1. Is this a valid assumption? I think its only true in the asymptotic limit of high N. On the other hand, it may he the only way to conduct the hypothesis test, so if N is too small for this assumption to be valid, then N may be too small for discovery.

`p = 1 - chi2.cdf(T, 1)`

Then its a one sided test so the significance is:

`Z = np.sqrt(chi2.ppf(1-2p, 1))`

If Z>=5 then we have 'discovered the signal' (success)

I will have a module for this: `hypothesis_test.py`

#### Statistical justification of the assumption

I think I need to justify "this test statistic, given the null hypothesis, is chi squared distributed with k=1"

This is Wilks' theorem, which states that under certain conditions, this test statistic follows a chi-squared distribution

In [9]:
from generation import generate_from_total_pdf
from distributions import total_cdf, total_model, background_model
from hypothesis_test import NP_test

true_params = {'f': 0.1, 'lam': 0.5, 'mu': 5.28, 'sigma': 0.018}

N_events = 100000
dataset = generate_from_total_pdf(N_events)

NP_test(dataset=dataset, cdf=total_cdf, bins=200, starting_params=true_params)


Generated 100000 events in 0.007047s
┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 0.6691 (chi2/ndof = 0.0)   │              Nfcn = 85               │
│ EDM = 1.83e-05 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │        No Parameters at limit        │
├──────────────────────────────────┼──────────────────────────────────────┤
│ Below EDM threshold (goal x 10)  │           Below call limit           │
├───────────────┬──────────────────┼───────────┬─────────────┬────────────┤
│  Covariance   │     Hesse ok     │ Accurate  │  Pos. def.  │ Not forced │
└───────────────┴──────────────────┴───────────┴─────────────┴────────────┘
┌───┬───────┬───────────┬───────────┬────────────┬─

(False, 4.59195039424652)

## Binomial dataset

After conducting hypothesis test on all datasets for a given $N_i$, we have essentailly conducted a Bernoulli trial, where the number of discoveries (successes), `r`, is binomially distributed, except the probability of success, $p_N$, is unknown. 

We estimate $\hat{p_i} = r/n$, where $n$ is the number of trials

And the standard error on this estimate is $\hat{\sigma_i} = \sqrt{\frac{\hat{p_i}(1-\hat{p_i})}{n_i}}$

Alternatively could we construct a confidence belt???

## p vs N

Once we have our dataset of $\{ N_i, \hat{p_i} \pm \hat{\sigma_i} \}$, we can plot $p$ vs $N$, fit a line of best fit.

We would need to justify the line we fit. Eg. if we fit a straight line, why would there be a linear relationship between $N$ and $p$?

The line of best fit should go through 68.3% of the confidence intervals. We will discuss whether this happens.

Predict $N_{discovery}$ (value of $N$ for $p=0.9$), as well as an uncertainty on $N_{discovery}$